SONG TRACKS ANALYSIS

Load packages

In [ ]:
library(MASS) #LDA
library(dplyr)
library(ggplot2)
library(car)
library(ca) #correspondence analysis
library(psych) 
library(caret) #cofusion martrix
library(cluster)    # clustering algorithms
library(factoextra) # clustering algorithms & visualization
library(purrr)
library(gridExtra) #visualize number of k clusters
library(class)
library(REdaS)
source("Confusion.R")

Unable to load all the packages in R jupyter notebook environment

Preprocessing

In [ ]:
infile1=read.csv('song_data.csv')
head(infile1)
bart_spher(infile1[2:15])
infile2=read.csv('song_info.csv')
head(infile2)
infile1$duration=round(infile1$song_duration_ms*1.66667e-5,3) #converting to minutes
infile1$artist=infile2$artist_name
nrow(infile1)
#remove duplicates
clean_infile1=distinct(infile1)
nrow(clean_infile1)
clean_infile1=subset(clean_infile1,select=-c(song_duration_ms))
#DISCRITIZE AUDIO VALENCE
summary(clean_infile1$audio_valence)
clean_infile1$valence[clean_infile1$audio_valence<=0.332]='Low'
clean_infile1$valence[clean_infile1$audio_valence<=0.7278 & clean_infile1$audio_valence>0.332]='Medium'
clean_infile1$valence[clean_infile1$audio_valence>0.7278]='High'

summary(clean_infile1$song_popularity)
clean_infile1$pop[clean_infile1$song_popularity>=63.75]='Very Popular'
clean_infile1$pop[clean_infile1$song_popularity<=37]='Very Not Popular'
clean_infile1$pop[clean_infile1$song_popularity>37 & clean_infile1$song_popularity<=52]='Not Popular'
clean_infile1$pop[clean_infile1$song_popularity>52 & clean_infile1$song_popularity<63.75]='Popular'
boxplot(clean_infile1$song_popularity~clean_infile1$valence,
        xlab='Valence',ylab='Popularity')
clean_infile1$pop=as.factor(clean_infile1$pop)
clean_infile1$valence=as.factor(clean_infile1$valence)

Validation setup

In [ ]:
smp_size <- floor(0.70 * nrow(clean_infile1))
set.seed(463)
train_ind <- sample(seq_len(nrow(clean_infile1)), size = smp_size)
train <- clean_infile1[train_ind, ]
test <- clean_infile1[-train_ind, ]
head(train)

CORRESPONDENCE ANALYSIS

In [ ]:
catable=table(clean_infile1$valence, clean_infile1$pop)
catable
margin.table(catable, 1) 
margin.table(catable, 2) 
round(prop.table(catable),4) # cell percentages
round(prop.table(catable, 1),4) # row percentages 
round(prop.table(catable, 2),4) # column percentages
#correspondance fit
fitca <- ca(catable)
mosaicplot(catable,shade=TRUE,main='Popularity vs. Valence')
print(fitca) # basic results 
summary(fitca) 
plot(fitca) # symmetric map
plot(fitca, mass = TRUE, contrib = "absolute", map =
       "rowgreen", arrows = c(FALSE, TRUE)) 

LINEAR DISCRIMINANT ANALYSIS

In [ ]:
ldatrain1=subset(train,select=-c(artist,duration,song_popularity,song_name,audio_mode,pop,audio_valence))
ldatest=subset(test,select=-c(artist,duration,song_popularity,song_name,audio_mode,pop,audio_valence))
vlda = lda(valence ~ ., data=ldatrain1)
vlda

#vlda$scaling #the loadings: how each variable contributes to the LD
#plot(vlda)
#reorder the loadings helps to interpret the results
print(vlda$scaling[order(vlda$scaling[,1]),])
#predict trianing values 
#use the scores to discriminate the classes
vlda.values = predict(vlda) #predict test set values
head(vlda.values$x)
head(ldatrain1)
compare=subset(ldatrain1,select=-c(valence))
compare$LD=vlda.values$x
head(compare)
round(cor(compare),2)

par(mar=c(4,4,4,4))
ldahist(data=vlda.values$x[, 1], g=ldatrain1$valence)
ldahist(data=vlda.values$x[, 2], g=ldatrain1$valence)
plot(vlda.values$x[, 1], vlda.values$x[, 2], col=ldatrain1$valence, pch=16)
ldatable=table(ldatrain1$valence, vlda.values$class)
confusion(vlda.values$class, ldatrain1$valence)
confusionMatrix(ldatable)

#test model performance on test set 
test.lda.values1 = predict(vlda,newdata =ldatest) #predict test set values
test.lda.values1$x
ldahist(data=test.lda.values1$x[, 1], g=test$valence)
ldahist(data=test.lda.values1$x[, 2], g=test$valence)
plot(test.lda.values1$x[, 1], test.lda.values1$x[, 2], col=test$valence, pch=16)
ldatab=table(test$valence, test.lda.values1$class)
confusionMatrix(ldatab)

Exploring "very popylar songs"

In [ ]:
#extracting subset
popsong=filter(clean_infile1,song_popularity>80) #examine songs have over 80 pop score
popsong=popsong[order(-popsong$song_popularity),]
nrow(distinct(popsong))
popsong=popsong %>% distinct(song_name,.keep_all=T)
nrow(popsong)
singers=popsong$artist
row.names(popsong)=popsong$song_name
head(popsong)
#for (i in singers)
#{x=length(which(clean_infile1$artist==i))
#cat(i,x,sep=':',fill=T)}
popsong$count=c(10,44,11,2,16,45,9,12,1,8,25,6,5,6,6,6,10,25,18,7,10,4,5,16,3,20,6,1,45,
                10,10,4,45,16,16,16,9,10,15,6,22,17,8,16,6,7,7,17,6,4,25,7,6,13,14,25,9,
                5,12,13,14,11,6,4,5,6,3,4,25,9,7,8,4,2,17,8,10,10,1,17,1,13,10,25,17,2,14,
                6,2,1,25,4,3,23,22,8,7,1,7,1,12,17,3,25,10,11,3,17,7,7,7,14,5,6,6,4,10,1,10,
                4,5,17,57,6,3,10,25,1,22,7,22,17,11,8,4,5,7,4,3,25,10,17,9,9,1,5,7,23,9,18,
                9,8,1,9,13,7,10,6,9,12,16,11,14,11,10,18,23,23,10,6,2,7,45,10,13,16,18,1,57,9,
                13,6,10,3,6,7,25,2,1,6,13,22,5,1,1,11,14,6,11,5,4,8,11,16,17,16,10,8,10,6,4,2,
                3,44,13,7,10,1,1,1,2,1,6,10,9,6,10,8,15,10,3,7,8,11,12,6,10,10,9,5,23,2,2,1,1,11,
                4,4,7,2,4,9,7,4,9,22,7,5,10,22,10,22,10,4,6,4,6,6,2,8,3,11,5,10,1,2,5,6,17,4,7,10,
                9,2,17,4,1,7,16,17,4,23,7,7,5,7,2,5,7,16,18,12,18,5,7,9,5,7,2,2,8,7,11,6,6,57,1,12,
                8,1,13,3,19,4,10,18,10,18,7,4,5,23,5,7,11,11,5,6,17,2,3,23,9,13,12,3,4,11,10,9,3,4,
                10,2,10,6,7,5,13,9,25,4,9,8,15,3,9,3,8,2,7,25,4,16,8,13,13,6,3,4,16,18,3,1,6,5,16,
                8,44,13,5,5,14,4,1,12,6,15,22,10,9,7,3,6,8,8,12,6,5,10,15,6,17,10,5,11,14,9,2,1,10,
                3,11,1)
cor(popsong$song_popularity,popsong$count)
summary(popsong) #will remove some variables because of commonalities
summary(popsong$count)
popsong=subset(popsong,select=-c(song_name,instrumentalness,liveness,audio_mode,time_signature,duration,pop))
head(popsong)
cor(popsong$song_popularity,popsong$audio_valence)

MULTI-DIMENSIONAL SCALING 

In [ ]:
hits=subset(popsong,select=-c(artist,valence,pop,song_popularity))
head(hits)
hits.dist <- dist(hits)
summary(hits.dist)
hits.mds <- isoMDS(hits.dist)
plot(hits.mds$points, type = "n")
text(hits.mds$points, labels = as.character(1:nrow(hits)))
hits.sh <- Shepard(hits.dist,hits.mds$points)
plot(hits.sh, pch = ".")
lines(hits.sh$x,hits.sh$yf, type = "S",col='red')

PRICIPAL COMPONENT ANALYSIS

In [ ]:
ph1=prcomp(hits[1:7],scale=T) #fit on all metric data
summary(ph1) #how much variance of the ds is explained; and cumulative proportion
plot(ph1) #knee plot to see how many pcs needed to explain at least 90%; agree with above?
abline(1,0,col='red')
#round(p1$rotation,2) #formula of each pc;% of each variable in each pc;components of each pc
pch1= principal(hits[1:7], rotate="varimax", nfactors=4, scores=TRUE)
print(pch1$loadings, cutoff=.4, sort=T)

COMMON FACTOR ANALYSIS

In [ ]:
fith=factanal(hits[1:7],3, rotation="varimax")
print(fith, digits=2, cutoff=.4, sort=TRUE)
# plot factor 1 by factor 2 
loadh=fith$loadings[,1:2] 
plot(loadh,type="n") # set up plot 
text(loadh,labels=names(hits),cex=.7) 

DENSITY CLUSTERING

In [ ]:
kNNdistplot(hits, k = 5)
abline(h=.5, col = "red", lty=2)
dens = dbscan(hits, eps=.5,MinPts = 10)
dens
pairs(hits, col = dens$cluster + 1L)